# CV07 - 세그멘테이션 살펴보기

세그멘테이션: 픽셀 단위에서 이미지의 각 부분이 어떤 의미를 갖는 영역인지 분리하는 방법  
이미지에서 영역을 분리하는 방식은 인스턴스 세그멘테이션과 시맨틱 세그멘테이션이 있다.  
시맨틱은 동일한 개체를 같이 묶어서 분리하는 반면 인스턴스는 동일한 개체도 각각 분류한다.  
#### 시맨틱 세그멘테이션(Semantic Segmentation)
U-Net구조: 대표적인 모델이며, 572x572 크기의 이미지를 입력하면 388x388 크기에 두 가지의  
클래스를 가진 세그멘테이션 맵이 나온다.  
https://mi.eng.cam.ac.uk/projects/segnet/#demo

#### 인스턴스 세그멘테이션(Instance Segmentation)
Mask R-CNN: 대표적인 모델이며, Faster R-CNN을 계승하였고 ROI Pooling layer  
개념을 개선하여 RolAlign, 클래스별 마스크 분리을 통해 클래스별 object detection,  
semantic segmentation을 하나의 task로 엮어냈다.

ROIPool의 한계점: 다양한 크기의 ROI영역을 같은 크기로 맞추려다보니 정보가 손실되거나  
왜곡되는 현상이 발생한다.
RoIAlign은 이런 한계점을 해결한다.  
https://towardsdatascience.com/understanding-region-of-interest-part-2-roi-align-and-roi-warp-f795196fc193  
bbox head: bounding box regression
mask head: 마스크 예측, 클래스

#### FCN
https://youtu.be/_52dopGu3Cw  
https://medium.com/@msmapark2/fcn-%EB%85%BC%EB%AC%B8-%EB%A6%AC%EB%B7%B0-fully-convolutional-networks-for-semantic-segmentation-81f016d76204  
https://arxiv.org/abs/1411.4038  

기본적인 VGG모델은 이미지 특성을 추출 후 마지막에 fully connected layer를 붙여서  
클래스별 확률을 계산하고 이미지를 분류한다.  
FCN은 fully connected layer 대신 CNN를 붙이는데 CNN은 이미지 내 위치 특성을  가지고 있기 때문이다. 클래스 단위의 히트맵을 얻을 수 있다. 이 CNN은 1x1의 커널 크기와  클래스의 갯수만큼 채널을 가진다.

히트맵의 크기는 작기 때문에 upsampling을 하여 사이즈를 키워준다.  
FCN은 deconvolution과 interpolation방식을 활용한다.  
skip architecture: upsampling한 맵에 앞 블록에서 얻은 예측 결과 맵을 더한 후 다시 upsampling하는 방법이다.

#### U-Net
https://www.youtube.com/watch?v=evPZI9B2LvQ  
https://medium.com/@msmapark2/u-net-%EB%85%BC%EB%AC%B8-%EB%A6%AC%EB%B7%B0-u-net-convolutional-networks-for-biomedical-image-segmentation-456d6901b28a  
https://arxiv.org/pdf/1505.04597.pdf  

전체 구조는 U자 형태이며 좌측에는 contracting path, 우측에는 Expansive path로 나뉜다.  
contracting path는 convolution network와 유사하며 3x3 convolution과 relu  
downsampling(2x2커널, 2 stride) Maxpooling, 마지막엔 convolution 채널 크기가  
두배가 되도록 설계  
Expansive path는 2x2 up-convolution과 3x3 convolution, 마지막에  
1x1 convolution을 사용하여 히트맵 출력  
572x572 -> 388x388 입출력 이미지 크기가 다른 것은 convolution연산시 패딩을  
하지 않기 때문이다.  

타일 기법: FCN은 입력 이미지의 크기를 조정하여 히트맵을 얻지만 U-Net은 타일 방식을  
사용하여 어느 정도 겹치는 구간으로 타일을 나누어 네트워크에 적용하기 때문에 높은 해상도  
를 얻을 수 잇다.

데이터 불균형 해결: 세포 검출을 할 때 경계 또한 예측해야 하는데 그 면적이 대단히 작기  
때문에 분포를 고려한 weight map를 사용한다. 여기에서 weight는 손실함수에 적용되는  
가중치를 말한다.

#### DeepLab
https://blog.lunit.io/2018/07/02/deeplab-v3-encoder-decoder-with-atrous-separable-convolution-for-semantic-image-segmentation/  
https://medium.com/hyunjulie/2%ED%8E%B8-%EB%91%90-%EC%A0%91%EA%B7%BC%EC%9D%98-%EC%A0%91%EC%A0%90-deeplab-v3-ef7316d4209d  
https://www.youtube.com/watch?v=JiC78rUF4iI  
https://arxiv.org/pdf/1802.02611.pdf  

전체 구조는 인코더와 디코더로 나눌 수 있고 인코더는 이미지에서 필요한 정보들의 특성을  
추출해내는 모듈이고 디코더는 추출된 특성을 이용해 원하는 정보를 예측하는 모듈이다.  
3x3 conv(U-Net) -> Atrous conv(Deeplab) 
Atrous conv를 여러 크기에 다양하게 적용한 것이 ASPP(Atrous Spatial Pyramid Pooling)

Atrous Convolution: stride을 하면서 꼼꼼히 보는 것이 아니라 간격을 두고 띄엄띄엄 보는 것  
https://zzsza.github.io/data/2018/02/23/introduction-convolution/  

Spatial Pyramid Pooling: 여러 가지 스케일로 conv와 pooling을 하고 나온 다양한 
특성을 연결해준다.  
여기서 conv를 atrous conv로 바꾸면 ASPP다.  
이러한 구조는 입력이미지의 크기와 관계없이 동일한 구조를 활용할 수 있다.  
https://yeomko.tistory.com/14  

#### 세그멘테이션의 평가
https://www.jeremyjordan.me/evaluating-image-segmentation-models/  
시맨틱 세그멘테이션의 결괏값은 이미지의 크기에 맞는 세그멘테이션 맵 크기, 시맨틱 클래스의 수에 맞는 채널 크기

픽셀별 정확도(Pixel Accuracy)